In [7]:
import pandas as pd

In [8]:
from sklearn.model_selection import train_test_split
from cnn import ChessDataset, ChessEvaluationConvolutionalNetwork, ConvolutionInputModel
from torch.utils.data import DataLoader
import torch


df = pd.read_csv('../../data/chess_data.csv')
df = df[~df['Evaluation'].str.contains('#', na=False)]

df = df.reset_index(drop=True)


df['board_vec'] = df.apply(lambda row: ConvolutionInputModel.fen_to_feature_array(row['FEN'])[0].squeeze(0), axis=1)
df['meta_vec'] = df.apply(lambda row: ConvolutionInputModel.fen_to_feature_array(row['FEN'])[1].squeeze(0), axis=1)

X = df[['board_vec', 'meta_vec']]
df['Evaluation'] = pd.to_numeric(df['Evaluation'], errors='raise')
Y = (df['Evaluation'] - df['Evaluation'].min()) / (df['Evaluation'].max() - df['Evaluation'].min())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Convert to a column tensor
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = ChessDataset(X_train, y_train_tensor)
test_dataset = ChessDataset(X_test, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)



In [9]:
import torch.nn as nn
import torch.optim as optim

batch_size = 32
epochs = 10
lr = 0.001


model = ChessEvaluationConvolutionalNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(epochs):
    loss = 0
    model.train()

    for x_board, x_meta, target in train_loader:
        optimizer.zero_grad()
        output = model(x_board, x_meta)
        loss = criterion(output, target)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Epoch Loss: {loss}")

Epoch 1/10, Epoch Loss: 0.001021010335534811
Epoch 2/10, Epoch Loss: 1.8663857190404087e-05
Epoch 3/10, Epoch Loss: 0.0008417324279434979
Epoch 4/10, Epoch Loss: 0.00023078141384758055
Epoch 5/10, Epoch Loss: 0.00013765107723884284
Epoch 6/10, Epoch Loss: 6.124500941950828e-05
Epoch 7/10, Epoch Loss: 0.00040205568075180054
Epoch 8/10, Epoch Loss: 2.9310069294297136e-05
Epoch 9/10, Epoch Loss: 0.0001724425092106685
Epoch 10/10, Epoch Loss: 9.784070425666869e-05


In [10]:
import numpy as np
model.eval()
test_loss = 0.0
with torch.no_grad():
    for x_board, x_meta, target in test_loader:
        outputs = model(x_board, x_meta)
        loss = criterion(outputs, target)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 0.0266


In [11]:
torch.save(model.state_dict(), 'convolutional_network.pth')